# Reachability Analysis


In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()
from IPython.display import HTML

In [2]:
import torch
from mantrap.agents import *

In [11]:
agent = DoubleIntegratorDTAgent(position=torch.zeros(2), velocity=torch.tensor([2, 3]))
boundary_double = agent.reachability_boundary(time_steps=5, dt=1.0).samples().detach().numpy()

In [8]:
agent = IntegratorDTAgent(position=torch.zeros(2), velocity=torch.tensor([2, 3]))
boundary_single = agent.reachability_boundary(time_steps=5, dt=1.0).samples().detach().numpy()

In [9]:
plt.plot(boundary_single[:, 0], boundary_single[:, 1], 'x', label="single")
plt.plot(boundary_double[:, 0], boundary_double[:, 1], 'o', label="double")
plt.axis('equal')
plt.legend()
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute '_t_horizon'